In [1]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

from helper_functions.preprocessing import replace_black_with_median, preprocess_image
from helper_functions.encode import prepare_training_data
from helper_functions.encode import CHARACTERS
from helper_functions.encode import IMG_HEIGHT, IMG_WIDTH
from helper_functions.models.resnet import HybridResNet50


# Paths
TRAIN_FOLDER_PATH = "../data/train/combine"
TEST_FOLDER_PATH = "../data/test/combine"

In [2]:
# CNN parameters
BATCH_SIZE = 64
EPOCHS = 15
VALIDATION_SPLIT = 0.2

In [3]:
img = cv2.imread('../data/train/combine/000s-0.png')
processed_img = preprocess_image(img)

cv2.imshow('image', processed_img)

# Wait for a key press (0 means indefinite wait)
cv2.waitKey(0)

# Close all OpenCV windows
cv2.destroyAllWindows()

# Load Training Data

In [4]:
print("Preparing training data...")
X_img, X_features, y, feature_names, scaler = prepare_training_data(TRAIN_FOLDER_PATH)

Preparing training data...


Preparing Data: 100%|█████████████████████████████████████████████████████████████| 7437/7437 [00:26<00:00, 281.34it/s]


# RESNET Model


In [5]:
model = HybridResNet50()

In [6]:
model(torch.from_numpy(X_img[0]).unsqueeze(0), torch.from_numpy(X_features[0]).unsqueeze(0))

torch.Size([1, 2112])


tensor([[ 0.0675, -0.1510,  0.0201,  0.1192, -0.0359,  0.0950,  0.0218,  0.1526,
          0.0178,  0.1455,  0.2446, -0.0127,  0.1388,  0.0890, -0.0866,  0.0148,
          0.1160,  0.0659,  0.0401, -0.0035, -0.0079, -0.1441, -0.0410,  0.0536,
          0.0270,  0.1114,  0.0870,  0.0884, -0.0385, -0.0176, -0.0660,  0.1162,
         -0.0840,  0.1277, -0.0627,  0.0174]], grad_fn=<AddmmBackward0>)